<a href="https://colab.research.google.com/github/Ki-i/APRiL/blob/master/kmeanscuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git config --global url."https://github.com".insteadOf git://github.com


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0j5lpm0f
  Running command git clone --filter=blob:none --quiet git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0j5lpm0f
  Resolved git://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=c791b80c52a77ffe7475c55dcc94b63d29e9ad5ff64a88dd3413ae96bf07b155
  Stored in directory: /tmp/pip-ephem-wheel-cache-5bsarbe0/wheels/31/f1/9c/d0fdd145ee4ae98a3bf97a02f5ade793299bc4a680d897163f
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin



created output directory at /content/src
Out bin /content/result.out


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### cuda test程序

In [5]:
%cd /content/drive/MyDrive/cuda_kmeans_data
!nvcc deviceQuery.cpp -o deviceQuary -I /content/drive/MyDrive/cuda_kmeans_data
!./deviceQuary

/content/drive/MyDrive/cuda_kmeans_data
./deviceQuary Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.0 / 11.8
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835398144 bytes)
MapSMtoCores for SM 7.5 is undefined.  Default to use 128 Cores/SM
MapSMtoCores for SM 7.5 is undefined.  Default to use 128 Cores/SM
  (40) Multiprocessors, (128) CUDA Cores/MP:     5120 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2

#### cuda加速kmeans

In [76]:
%cd /content/drive/MyDrive/CUDAhw


/content/drive/MyDrive/CUDAhw


In [63]:
!nvcc -c cuda_kmeans.cu -o cuda_kmeans.o
!nvcc -c cuda_main.cu -o cuda_main.o
!nvcc cuda_kmeans.o cuda_main.o -o cuda_kmeans_main -I /content/drive/MyDrive/CUDAhw
!/content/drive/MyDrive/CUDAhw/cuda_kmeans_main

cuda_main.cu(24): warning #177-D: variable "i" was declared but never referenced

cuda_main.cu(24): warning #177-D: variable "j" was declared but never referenced

cuda_main.cu(25): warning #177-D: variable "opt" was declared but never referenced

cuda_main.cu(28): warning #550-D: variable "clusters" was set but never used

cuda_main.cu(29): warning #177-D: variable "threshold" was declared but never referenced

cuda_main.cu(101): warning #177-D: variable "elapsedTime" was declared but never referenced

标签文件读取结束读取结束
已完成0行读取
读取结束
文件读取时间：1561793ms
完成分类

Running Time：1295943ms
正确图像个数1.000000
准确率为0.001000

In [77]:
!nvcc -c seq_kmeans.c -o seq_kmeans.o
!nvcc -c seq_main.c -o seq_main.o
!nvcc seq_kmeans.o seq_main.o -o seq_kmeans_main -I /content/drive/MyDrive/CUDAhw
!/content/drive/MyDrive/CUDAhw/seq_kmeans_main

seq_main.c: In function ‘main’:
seq_main.c:32:15: warning: implicit declaration of function ‘malloc’ [-Wimplicit-function-declaration]
   32 | pred = (int*) malloc(numObjs * sizeof(int));
      |               ^~~~~~
seq_main.c:32:15: warning: incompatible implicit declaration of built-in function ‘malloc’
seq_main.c:5:1: note: include ‘<stdlib.h>’ or provide a declaration of ‘malloc’
    4 | #include "kmeans.h"
  +++ |+#include <stdlib.h>
    5 | 
标签文件读取结束读取结束
已完成0行读取
读取结束
文件读取时间：1579585ms
完成分类

Running Time：8423040ms
正确图像个数1.000000
准确率为0.001000

In [ ]:
%cd /content/drive/MyDrive/cuda_kmeans_data
!nvcc deviceQuery.cpp -o deviceQuary -I /content/drive/MyDrive/cuda_kmeans_data
!./deviceQuary

In [43]:
%cd /content/drive/MyDrive/CUDAhw

/content/drive/MyDrive/CUDAhw


In [45]:

!nvcc test.cu -o test -I /content/drive/MyDrive/CUDAhw
!./test

test.cu(384): warning #177-D: variable "i" was declared but never referenced

test.cu(384): warning #177-D: variable "j" was declared but never referenced

test.cu(385): warning #177-D: variable "opt" was declared but never referenced

test.cu(388): warning #550-D: variable "clusters" was set but never used

test.cu(389): warning #177-D: variable "threshold" was declared but never referenced

test.cu(461): warning #177-D: variable "elapsedTime" was declared but never referenced

Failed to open file


##### cuda并行kmeans

In [46]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<cuda_runtime.h>
#include <assert.h>

#include<stdio.h>
#include<cuda.h>
#include<cuda_runtime.h>
#include <time.h>



#define NUMTHREADSPERCLUSTERBLOCK 512
#define BLOCK_NUM 32   //块数量
#define THREAD_NUM 256 // 每个块中的线程数
#define LOOP_N BLOCK_NUM * THREAD_NUM * 1000000


#define msg(format, ...) do { fprintf(stderr, format, ##__VA_ARGS__); } while (0)
#define err(format, ...) do { fprintf(stderr, format, ##__VA_ARGS__); exit(1); } while (0)

#define malloc2D(name, xDim, yDim, type) do {               \
    name = (type **)malloc(xDim * sizeof(type *));          \
    assert(name != NULL);                                   \
    name[0] = (type *)malloc(xDim * yDim * sizeof(type));   \
    assert(name[0] != NULL);                                \
    for (size_t i = 1; i < xDim; i++)                       \
        name[i] = name[i-1] + yDim;                         \
} while (0)

#ifdef __CUDACC__
inline void checkCuda(cudaError_t e) {
    if (e != cudaSuccess) {
        // cudaGetErrorString() isn't always very helpful. Look up the error
        // number in the cudaError enum in driver_types.h in the CUDA includes
        // directory for a better explanation.
        err("CUDA Error %d: %s\n", e, cudaGetErrorString(e));
    }
}

inline void checkLastCudaError() {
    checkCuda(cudaGetLastError());
}
#endif

__global__ void leib_pi(double* g_sum) {
    const int tid = threadIdx.x;
    const int bid = blockIdx.x;
    double tmp = 0;
    int flag = -1;
    int idx = bid * THREAD_NUM + tid;
    int start = idx * 100000 + 1;
    int end = start + 100000;
    for (int i = start; i < end; i++) {
        tmp += flag * (1./(2 * i + 1));
        flag = -flag;
    }
    g_sum[bid*THREAD_NUM+tid] = tmp;
}


static inline int nextPowerOfTwo(int n) {
    n--;

    n = n >>  1 | n;
    n = n >>  2 | n;
    n = n >>  4 | n;
    n = n >>  8 | n;
    n = n >> 16 | n;
//  n = n >> 32 | n;    //  For 64-bit ints

    return ++n;
}

__host__ __device__ inline static
float euclid_dist_2(int    numCoords,
                    int    numObjs,
                    int    numClusters,
                    float *objects,     // [numCoords][numObjs]
                    float *clusters,    // [numCoords][numClusters]
                    int    objectId,
                    int    clusterId)
{
    int i;
    float ans=0.0;

    for (i = 0; i < numCoords; i++) {
        ans += (objects[numObjs * i + objectId] - clusters[numClusters * i + clusterId]) *
               (objects[numObjs * i + objectId] - clusters[numClusters * i + clusterId]);
    }

    return(ans);
}

__global__ static
void find_nearest_cluster(int numCoords,
                          int numObjs,
                          int numClusters,
                          float *objects,           //  [numCoords][numObjs]
                          float *deviceClusters,    //  [numCoords][numClusters]
                          int *membership,          //  [numObjs]
                          int *intermediates)
{
    extern __shared__ char sharedMemory[];

    //  The type chosen for membershipChanged must be large enough to support
    //  reductions! There are blockDim.x elements, one for each thread in the
    //  block. See numThreadsPerClusterBlock in cuda_kmeans().
    unsigned char *membershipChanged = (unsigned char *)sharedMemory;
#if BLOCK_SHARED_MEM_OPTIMIZATION
    float *clusters = (float *)(sharedMemory + blockDim.x);
#else
    float *clusters = deviceClusters;
#endif

    membershipChanged[threadIdx.x] = 0;

#if BLOCK_SHARED_MEM_OPTIMIZATION
    //  BEWARE: We can overrun our shared memory here if there are too many
    //  clusters or too many coordinates! For reference, a Tesla C1060 has 16
    //  KiB of shared memory per block, and a GeForce GTX 480 has 48 KiB of
    //  shared memory per block.
    for (int i = threadIdx.x; i < numClusters; i += blockDim.x) {
        for (int j = 0; j < numCoords; j++) {
            clusters[numClusters * j + i] = deviceClusters[numClusters * j + i];
        }
    }
    __syncthreads();
#endif

    int objectId = blockDim.x * blockIdx.x + threadIdx.x;

    if (objectId < numObjs) {
        int   index, i;
        float dist, min_dist;

        /* find the cluster id that has min distance to object */
        index    = 0;
        min_dist = euclid_dist_2(numCoords, numObjs, numClusters,
                                 objects, clusters, objectId, 0);

        for (i=1; i<numClusters; i++) {
            dist = euclid_dist_2(numCoords, numObjs, numClusters,
                                 objects, clusters, objectId, i);
            /* no need square root */
            if (dist < min_dist) { /* find the min and its array index */
                min_dist = dist;
                index    = i;
            }
        }

        if (membership[objectId] != index) {
            membershipChanged[threadIdx.x] = 1;
        }

        /* assign the membership to object objectId */
        membership[objectId] = index;

        __syncthreads();    //  For membershipChanged[]

        //  blockDim.x *must* be a power of two!
        for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
            if (threadIdx.x < s) {
                membershipChanged[threadIdx.x] +=
                    membershipChanged[threadIdx.x + s];
            }
            __syncthreads();
        }

        if (threadIdx.x == 0) {
            intermediates[blockIdx.x] = membershipChanged[0];
        }
    }
}

__global__ static
void compute_delta(int *deviceIntermediates,
                   int numIntermediates,    //  The actual number of intermediates
                   int numIntermediates2)   //  The next power of two
{
    //  The number of elements in this array should be equal to
    //  numIntermediates2, the number of threads launched. It *must* be a power
    //  of two!
    extern __shared__ unsigned int intermediates[];

    //  Copy global intermediate values into shared memory.
    intermediates[threadIdx.x] =
        (threadIdx.x < numIntermediates) ? deviceIntermediates[threadIdx.x] : 0;

    __syncthreads();

    //  numIntermediates2 *must* be a power of two!
    for (unsigned int s = numIntermediates2 / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            intermediates[threadIdx.x] += intermediates[threadIdx.x + s];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        deviceIntermediates[0] = intermediates[0];
    }
}

float** cuda_kmeans(float **objects,      /* in: [numObjs][numCoords] */
                   int     numCoords,    /* no. features */
                   int     numObjs,      /* no. objects */
                   int     numClusters,  /* no. clusters */
                   float   threshold,    /* % objects change membership */
                   int    *membership,   /* out: [numObjs] */
                   int    *loop_iterations)
{
    int      i, j, index, loop=0;
    int     *newClusterSize; /* [numClusters]: no. objects assigned in each
                                new cluster */
    float    delta;          /* % of objects change their clusters */
    float  **dimObjects;
    float  **clusters;       /* out: [numClusters][numCoords] */
    float  **dimClusters;
    float  **newClusters;    /* [numCoords][numClusters] */

    float *deviceObjects;
    float *deviceClusters;
    int *deviceMembership;
    int *deviceIntermediates;

    //  Copy objects given in [numObjs][numCoords] layout to new
    //  [numCoords][numObjs] layout
    malloc2D(dimObjects, numCoords, numObjs, float);
    for (i = 0; i < numCoords; i++) {
        for (j = 0; j < numObjs; j++) {
            dimObjects[i][j] = objects[j][i];
        }
    }

    /* pick first numClusters elements of objects[] as initial cluster centers*/
    malloc2D(dimClusters, numCoords, numClusters, float);
    for (i = 0; i < numCoords; i++) {
        for (j = 0; j < numClusters; j++) {
            dimClusters[i][j] = dimObjects[i][j];
        }
    }

    /* initialize membership[] */
    for (i=0; i<numObjs; i++) membership[i] = -1;

    /* need to initialize newClusterSize and newClusters[0] to all 0 */
    newClusterSize = (int*) calloc(numClusters, sizeof(int));
    assert(newClusterSize != NULL);

    malloc2D(newClusters, numCoords, numClusters, float);
    memset(newClusters[0], 0, numCoords * numClusters * sizeof(float));

    //  To support reduction, numThreadsPerClusterBlock *must* be a power of
    //  two, and it *must* be no larger than the number of bits that will
    //  fit into an unsigned char, the type used to keep track of membership
    //  changes in the kernel.
    const unsigned int numThreadsPerClusterBlock = NUMTHREADSPERCLUSTERBLOCK;
    const unsigned int numClusterBlocks =
        (numObjs + numThreadsPerClusterBlock - 1) / numThreadsPerClusterBlock;
#if BLOCK_SHARED_MEM_OPTIMIZATION
    const unsigned int clusterBlockSharedDataSize =
        numThreadsPerClusterBlock * sizeof(unsigned char) +
        numClusters * numCoords * sizeof(float);

    cudaDeviceProp deviceProp;
    int deviceNum;
    cudaGetDevice(&deviceNum);
    cudaGetDeviceProperties(&deviceProp, deviceNum);

    if (clusterBlockSharedDataSize > deviceProp.sharedMemPerBlock) {
        err("WARNING: Your CUDA hardware has insufficient block shared memory. "
            "You need to recompile with BLOCK_SHARED_MEM_OPTIMIZATION=0. "
            "See the README for details.\n");
    }
#else
    const unsigned int clusterBlockSharedDataSize =
        numThreadsPerClusterBlock * sizeof(unsigned char);
#endif

    const unsigned int numReductionThreads =
        nextPowerOfTwo(numClusterBlocks);
    const unsigned int reductionBlockSharedDataSize =
        numReductionThreads * sizeof(unsigned int);

    checkCuda(cudaMalloc(&deviceObjects, numObjs*numCoords*sizeof(float)));
    checkCuda(cudaMalloc(&deviceClusters, numClusters*numCoords*sizeof(float)));
    checkCuda(cudaMalloc(&deviceMembership, numObjs*sizeof(int)));
    checkCuda(cudaMalloc(&deviceIntermediates, numReductionThreads*sizeof(unsigned int)));

    checkCuda(cudaMemcpy(deviceObjects, dimObjects[0],
              numObjs*numCoords*sizeof(float), cudaMemcpyHostToDevice));
    checkCuda(cudaMemcpy(deviceMembership, membership,
              numObjs*sizeof(int), cudaMemcpyHostToDevice));

    do {
        checkCuda(cudaMemcpy(deviceClusters, dimClusters[0],
                  numClusters*numCoords*sizeof(float), cudaMemcpyHostToDevice));

        find_nearest_cluster
            <<< numClusterBlocks, numThreadsPerClusterBlock, clusterBlockSharedDataSize >>>
            (numCoords, numObjs, numClusters,
             deviceObjects, deviceClusters, deviceMembership, deviceIntermediates);

        cudaDeviceSynchronize(); checkLastCudaError();

        compute_delta <<< 1, numReductionThreads, reductionBlockSharedDataSize >>>
            (deviceIntermediates, numClusterBlocks, numReductionThreads);

        cudaDeviceSynchronize(); checkLastCudaError();

        int d;
        checkCuda(cudaMemcpy(&d, deviceIntermediates,
                  sizeof(int), cudaMemcpyDeviceToHost));
        delta = (float)d;

        checkCuda(cudaMemcpy(membership, deviceMembership,
                  numObjs*sizeof(int), cudaMemcpyDeviceToHost));

        for (i=0; i<numObjs; i++) {
            /* find the array index of nestest cluster center */
            index = membership[i];

            /* update new cluster centers : sum of objects located within */
            newClusterSize[index]++;
            for (j=0; j<numCoords; j++)
                newClusters[j][index] += objects[i][j];
        }

        //  TODO: Flip the nesting order
        //  TODO: Change layout of newClusters to [numClusters][numCoords]
        /* average the sum and replace old cluster centers with newClusters */
        for (i=0; i<numClusters; i++) {
            for (j=0; j<numCoords; j++) {
                if (newClusterSize[i] > 0)
                    dimClusters[j][i] = newClusters[j][i] / newClusterSize[i];
                newClusters[j][i] = 0.0;   /* set back to 0 */
            }
            newClusterSize[i] = 0;   /* set back to 0 */
        }

        delta /= numObjs;
    } while (delta > threshold && loop++ < 500);

    *loop_iterations = loop + 1;

    /* allocate a 2D space for returning variable clusters[] (coordinates
       of cluster centers) */
    malloc2D(clusters, numClusters, numCoords, float);
    for (i = 0; i < numClusters; i++) {
        for (j = 0; j < numCoords; j++) {
            clusters[i][j] = dimClusters[j][i];
        }
    }

    checkCuda(cudaFree(deviceObjects));
    checkCuda(cudaFree(deviceClusters));
    checkCuda(cudaFree(deviceMembership));
    checkCuda(cudaFree(deviceIntermediates));

    free(dimObjects[0]);
    free(dimObjects);
    free(dimClusters[0]);
    free(dimClusters);
    free(newClusters[0]);
    free(newClusters);
    free(newClusterSize);

    return clusters;
}


#define numCoords 128*128
#define numObjs 1000
#define numClusters 8

float imgvec[numObjs][numCoords];

int label[numObjs];
short shortimgvec[numObjs][numCoords];

int main(){
int i,j;
int opt;
int    *pred;    /* [numObjs] */
float **objects;       /* [numObjs][numCoords] data objects */
float **clusters;      /* [numClusters][numCoords] cluster center */
float   threshold;
int     loop_iterations;
int begintime,endtime;


pred = (int*) malloc(numObjs * sizeof(int));
objects = (float**) malloc(numObjs * sizeof(float*));

begintime=clock();
FILE *fp;
fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128_label.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }

        for (int j = 0; j < numObjs; j++) {
            if (fscanf(fp, "%d", &label[j]) != 1) {
                printf("Failed to read matrix element\n");
                printf("%d",j);
                return 1;
            }
            if (j < numObjs - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
printf("标签文件读取结束读取结束\n");
fclose(fp);
short temp;
    fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }
    for (int i = 0; i < numObjs; i++) {
        for (int j = 0; j < numCoords; j++) {
            if (fscanf(fp, "%hd", &temp) != 1) {
                
                printf("Failed to read matrix element\n");
                printf("%d,%d",i,j);
                return 1;
            }
            imgvec[i][j]=(float)(temp);
            if (j < numCoords - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
        if(i%1000==0) printf("已完成%d行读取\n",i);
    }
    printf("读取结束\n");
    fclose(fp);
    endtime = clock();	//计时结束
	  printf("文件读取时间：%dms\n", endtime-begintime);
    for(int i=0;i<numObjs;i++){
        objects[i]=(float*)&(imgvec[i]);
    }

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    float elapsedTime=0;
    
	  begintime=clock();	//计时开始
    //cudaEventElapsedTime(&elapsedTime, start, stop);

    clusters = cuda_kmeans(objects, numCoords, numObjs, numClusters, 0.00001,
                    pred, &loop_iterations);
           
    //printf("所用时间为：%f\n <ms>",  elapsedTime);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
           
    printf("完成分类");
    endtime = clock();	//计时结束
	  printf("\n\nRunning Time：%dms\n", endtime-begintime);

    int sum0=0;
    int sum1=0;
    int sum01=0;
    int sum11=0;
    int positive,negative;
    for(int i=0;i<numObjs;i++){
        if(pred[i]==0){
            sum0++;
            if(label[i]==1) sum01++;
        }
        if(pred[i]==1){
            sum1++;
            if(label[i]==1) sum11++;
        }
    }

    if(sum01<sum0/2) positive=0;
    else positive=1;

    if(sum11<sum1/2) negative=0;
    else negative=1;

    float acc=0.0;
    for(int i=0;i<numObjs;i++){
        if(pred[i]==positive){
            if(label[i]==1) acc++;
        }
        if(pred[i]==negative){
            if(label[i]==0) acc++;
        }
    }
    printf("正确图像个数%f\n",acc);
    acc=acc/numObjs;
    printf("准确率为%f",acc);


}


Failed to open file



##### 串行kmeans

In [7]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<cuda_runtime.h>
#include <assert.h>

#include<stdio.h>
#include<cuda.h>
#include<cuda_runtime.h>
#include <time.h>




__host__ __device__ inline static
float euclid_dist_2(int    numdims,  /* no. dimensions */
                    float *coord1,   /* [numdims] */
                    float *coord2)   /* [numdims] */
{
    int i;
    float ans=0.0;

    for (i=0; i<numdims; i++)
        ans += (coord1[i]-coord2[i]) * (coord1[i]-coord2[i]);

    return(ans);
}

__host__ __device__ inline static
int find_nearest_cluster(int     numClusters, /* no. clusters */
                         int     numCoords,   /* no. coordinates */
                         float  *object,      /* [numCoords] */
                         float **clusters)    /* [numClusters][numCoords] */
{
    int   index, i;
    float dist, min_dist;

    index    = 0;
    min_dist = euclid_dist_2(numCoords, object, clusters[0]);

    for (i=1; i<numClusters; i++) {
        dist = euclid_dist_2(numCoords, object, clusters[i]);
        /* no need square root */
        if (dist < min_dist) { /* find the min and its array index */
            min_dist = dist;
            index    = i;
        }
    }
    return(index);
}

__host__ __device__ inline static
float** seq_kmeans(float **objects,      /* in: [numObjs][numCoords] */
                   int     numCoords,    /* no. features */
                   int     numObjs,      /* no. objects */
                   int     numClusters,  /* no. clusters */
                   float   threshold,    /* % objects change membership */
                   int    *membership,   /* out: [numObjs] */
                   int    *loop_iterations)
{
    int      i, j, index, loop=0;
    int     *newClusterSize; /* [numClusters]: no. objects assigned in each
                                new cluster */
    float    delta;          /* % of objects change their clusters */
    float  **clusters;       /* out: [numClusters][numCoords] */
    float  **newClusters;    /* [numClusters][numCoords] */

    /* allocate a 2D space for returning variable clusters[] (coordinates
       of cluster centers) */
    clusters    = (float**) malloc(numClusters *             sizeof(float*));
    // assert(clusters != NULL);
    clusters[0] = (float*)  malloc(numClusters * numCoords * sizeof(float));
    // assert(clusters[0] != NULL);
    for (i=1; i<numClusters; i++)
        clusters[i] = clusters[i-1] + numCoords;

    /* pick first numClusters elements of objects[] as initial cluster centers*/
    for (i=0; i<numClusters; i++)
        for (j=0; j<numCoords; j++)
            clusters[i][j] = objects[i][j];

    /* initialize membership[] */
    for (i=0; i<numObjs; i++) membership[i] = -1;

    /* need to initialize newClusterSize and newClusters[0] to all 0 */
    newClusterSize = (int*) calloc(numClusters, sizeof(int));
    // assert(newClusterSize != NULL);

    newClusters    = (float**) malloc(numClusters *            sizeof(float*));
    // assert(newClusters != NULL);
    newClusters[0] = (float*)  calloc(numClusters * numCoords, sizeof(float));
    // assert(newClusters[0] != NULL);
    for (i=1; i<numClusters; i++)
        newClusters[i] = newClusters[i-1] + numCoords;

    do {
        delta = 0.0;
        for (i=0; i<numObjs; i++) {
            /* find the array index of nestest cluster center */
            index = find_nearest_cluster(numClusters, numCoords, objects[i],
                                         clusters);

            /* if membership changes, increase delta by 1 */
            if (membership[i] != index) delta += 1.0;

            /* assign the membership to object i */
            membership[i] = index;

            /* update new cluster centers : sum of objects located within */
            newClusterSize[index]++;
            for (j=0; j<numCoords; j++)
                newClusters[index][j] += objects[i][j];
        }

        /* average the sum and replace old cluster centers with newClusters */
        for (i=0; i<numClusters; i++) {
            for (j=0; j<numCoords; j++) {
                if (newClusterSize[i] > 0)
                    clusters[i][j] = newClusters[i][j] / newClusterSize[i];
                newClusters[i][j] = 0.0;   /* set back to 0 */
            }
            newClusterSize[i] = 0;   /* set back to 0 */
        }
            
        delta /= numObjs;
        //printf("完成迭代%d,delta=%f\n",loop,delta);
        int sum1=0;
        int sum2=0;
        for(int i=0;i<numObjs;i++){
        if(membership[i]==0) sum1++;
        if(membership[i]==1) sum2++;
    }
    //printf("label0=%d，",sum1);
    //printf("label1=%d\n",sum2);
    } while (delta > threshold && loop++ < 500);

    *loop_iterations = loop + 1;

    free(newClusters[0]);
    free(newClusters);
    free(newClusterSize);

    return clusters;
}



#define numClusters 8
#define numCoords 128*128
#define numObjs 1000
float imgvec[numObjs][numCoords];



int label[numObjs];

int main(){
int i,j;
int opt;
int    *pred;    /* [numObjs] */
float **objects;       /* [numObjs][numCoords] data objects */
float **clusters;      /* [numClusters][numCoords] cluster center */
float   threshold;
int     loop_iterations;
int begintime,endtime;


pred = (int*) malloc(numObjs * sizeof(int));
objects = (float**) malloc(numObjs * sizeof(float*));

begintime=clock();
FILE *fp;
fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128_label.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }

        for (int j = 0; j < numObjs; j++) {
            if (fscanf(fp, "%d", &label[j]) != 1) {
                printf("Failed to read matrix element\n");
                printf("%d",j);
                return 1;
            }
            if (j < numObjs - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
printf("标签文件读取结束读取结束\n");
fclose(fp);
short temp;
    fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }
    for (int i = 0; i < numObjs; i++) {
        for (int j = 0; j < numCoords; j++) {
            if (fscanf(fp, "%hd", &temp) != 1) {
                
                printf("Failed to read matrix element\n");
                printf("%d,%d",i,j);
                return 1;
            }
            imgvec[i][j]=(float)(temp);
            if (j < numCoords - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
        if(i%1000==0) printf("已完成%d行读取\n",i);
    }
    printf("读取结束\n");
    fclose(fp);
  endtime = clock();	//计时结束
	  printf("文件读取时间：%dms\n", endtime-begintime);
    for(int i=0;i<numObjs;i++){
        objects[i]=(float*)&(imgvec[i]);
    }

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);

    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    
	  begintime=clock();	//计时开始


    clusters = seq_kmeans(objects, numCoords, numObjs, numClusters, 0.00001,
                    pred, &loop_iterations);
           
  
           
    printf("完成分类");
    endtime = clock();	//计时结束
	  printf("\n\nRunning Time：%dms\n", endtime-begintime);

    int sum0=0;
    int sum1=0;
    int sum01=0;
    int sum11=0;
    int positive,negative;
    for(int i=0;i<numObjs;i++){
        if(pred[i]==0){
            sum0++;
            if(label[i]==1) sum01++;
        }
        if(pred[i]==1){
            sum1++;
            if(label[i]==1) sum11++;
        }
    }

    if(sum01<sum0/2) positive=0;
    else positive=1;

    if(sum11<sum1/2) negative=0;
    else negative=1;

    float acc=0.0;
    for(int i=0;i<numObjs;i++){
        if(pred[i]==positive){
            if(label[i]==1) acc++;
        }
        if(pred[i]==negative){
            if(label[i]==0) acc++;
        }
    }
    printf("正确图像个数%f\n",acc);
    acc=acc/numObjs;
    printf("准确率为%f",acc);


}


标签文件读取结束读取结束
已完成0行读取
读取结束
文件读取时间：1678845ms
完成分类

Running Time：8820618ms
正确图像个数1.000000
准确率为0.001000


##### final版本


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<cuda_runtime.h>

#define BLOCK_NUM 32   //块数量
#define THREAD_NUM 256 // 每个块中的线程数
#define LOOP_N BLOCK_NUM * THREAD_NUM * 1000000


#define numCoords 128*128
#define numObjs 12058
#define numClusters 2

float imgvec[numObjs][numCoords];

int label[numObjs];
short shortimgvec[numObjs][numCoords];
clock_t* time

__global__ static void sumOfSquares(int *num, int* result, clock_t* time)

{
int sum = 0;
int i;
clock_t start = clock( );
for (i = 0; i < DATA_SIZE; i++) {
sum += num[i] * num[i] * num[i];}
*result = sum;
*time = clock( ) - start;
}


//距离计算函数
__host__ __device__ inline static
float euclid_dist_2(int    numCoords,
                    int    numObjs,
                    int    numClusters,
                    float *objects,     // [numCoords][numObjs]
                    float *clusters,    // [numCoords][numClusters]
                    int    objectId,
                    int    clusterId)
{
    int i;
    float ans=0.0;

    for (i = 0; i < numCoords; i++) {
        ans += (objects[numObjs * i + objectId] - clusters[numClusters * i + clusterId]) *
               (objects[numObjs * i + objectId] - clusters[numClusters * i + clusterId]);
    }

    return(ans);
}

//寻找距离最近的簇函数
__global__ static
void find_nearest_cluster(int numCoords,int numObjs,int numClusters,
                          float *objects,           //  [numCoords][numObjs]
                          float *deviceClusters,    //  [numCoords][numClusters]
                          int *membership,          //  [numObjs]
                          int *intermediates)
{
    extern __shared__ char sharedMemory[];

    //  The type chosen for membershipChanged must be large enough to support
    //  reductions! There are blockDim.x elements, one for each thread in the
    //  block. See numThreadsPerClusterBlock in cuda_kmeans().
    unsigned char *membershipChanged = (unsigned char *)sharedMemory;
#if BLOCK_SHARED_MEM_OPTIMIZATION
    float *clusters = (float *)(sharedMemory + blockDim.x);
#else
    float *clusters = deviceClusters;
#endif

    membershipChanged[threadIdx.x] = 0;

#if BLOCK_SHARED_MEM_OPTIMIZATION
    //  BEWARE: We can overrun our shared memory here if there are too many
    //  clusters or too many coordinates! For reference, a Tesla C1060 has 16
    //  KiB of shared memory per block, and a GeForce GTX 480 has 48 KiB of
    //  shared memory per block.
    for (int i = threadIdx.x; i < numClusters; i += blockDim.x) {
        for (int j = 0; j < numCoords; j++) {
            clusters[numClusters * j + i] = deviceClusters[numClusters * j + i];
        }
    }
    __syncthreads();
#endif

    int objectId = blockDim.x * blockIdx.x + threadIdx.x;

    if (objectId < numObjs) {
        int   index, i;
        float dist, min_dist;

        /* find the cluster id that has min distance to object */
        index    = 0;
        min_dist = euclid_dist_2(numCoords, numObjs, numClusters,
                                 objects, clusters, objectId, 0);

        for (i=1; i<numClusters; i++) {
            dist = euclid_dist_2(numCoords, numObjs, numClusters,
                                 objects, clusters, objectId, i);
            /* no need square root */
            if (dist < min_dist) { /* find the min and its array index */
                min_dist = dist;
                index    = i;
            }
        }

        if (membership[objectId] != index) {
            membershipChanged[threadIdx.x] = 1;
        }

        /* assign the membership to object objectId */
        membership[objectId] = index;

        __syncthreads();    //  For membershipChanged[]

        //  blockDim.x *must* be a power of two!
        for (unsigned int s = blockDim.x / 2; s > 0; s >>= 1) {
            if (threadIdx.x < s) {
                membershipChanged[threadIdx.x] +=
                    membershipChanged[threadIdx.x + s];
            }
            __syncthreads();
        }

        if (threadIdx.x == 0) {
            intermediates[blockIdx.x] = membershipChanged[0];
        }
    }
}
//计算误差
__global__ static
void compute_delta(int *deviceIntermediates,
                   int numIntermediates,    //  The actual number of intermediates
                   int numIntermediates2)   //  The next power of two
{
    //  The number of elements in this array should be equal to
    //  numIntermediates2, the number of threads launched. It *must* be a power
    //  of two!
    extern __shared__ unsigned int intermediates[];

    //  Copy global intermediate values into shared memory.
    intermediates[threadIdx.x] =
        (threadIdx.x < numIntermediates) ? deviceIntermediates[threadIdx.x] : 0;

    __syncthreads();

    //  numIntermediates2 *must* be a power of two!
    for (unsigned int s = numIntermediates2 / 2; s > 0; s >>= 1) {
        if (threadIdx.x < s) {
            intermediates[threadIdx.x] += intermediates[threadIdx.x + s];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        deviceIntermediates[0] = intermediates[0];
    }
}

float** cuda_kmeans(float **objects,      /* in: [numObjs][numCoords] */
                   int     numCoords,    /* no. features */
                   int     numObjs,      /* no. objects */
                   int     numClusters,  /* no. clusters */
                   float   threshold,    /* % objects change membership */
                   int    *membership,   /* out: [numObjs] */
                   int    *loop_iterations)
{
    int      i, j, index, loop=0;
    int     *newClusterSize; /* [numClusters]: no. objects assigned in each
                                new cluster */
    float    delta;          /* % of objects change their clusters */
    float  **dimObjects;
    float  **clusters;       /* out: [numClusters][numCoords] */
    float  **dimClusters;
    float  **newClusters;    /* [numCoords][numClusters] */

    float *deviceObjects;
    float *deviceClusters;
    int *deviceMembership;
    int *deviceIntermediates;

    //  Copy objects given in [numObjs][numCoords] layout to new
    //  [numCoords][numObjs] layout
    malloc2D(dimObjects, numCoords, numObjs, float);
    for (i = 0; i < numCoords; i++) {
        for (j = 0; j < numObjs; j++) {
            dimObjects[i][j] = objects[j][i];
        }
    }

    /* pick first numClusters elements of objects[] as initial cluster centers*/
    malloc2D(dimClusters, numCoords, numClusters, float);
    for (i = 0; i < numCoords; i++) {
        for (j = 0; j < numClusters; j++) {
            dimClusters[i][j] = dimObjects[i][j];
        }
    }

    /* initialize membership[] */
    for (i=0; i<numObjs; i++) membership[i] = -1;

    /* need to initialize newClusterSize and newClusters[0] to all 0 */
    newClusterSize = (int*) calloc(numClusters, sizeof(int));
    assert(newClusterSize != NULL);

    malloc2D(newClusters, numCoords, numClusters, float);
    memset(newClusters[0], 0, numCoords * numClusters * sizeof(float));

    //  To support reduction, numThreadsPerClusterBlock *must* be a power of
    //  two, and it *must* be no larger than the number of bits that will
    //  fit into an unsigned char, the type used to keep track of membership
    //  changes in the kernel.
    const unsigned int numThreadsPerClusterBlock = 128;
    const unsigned int numClusterBlocks =
        (numObjs + numThreadsPerClusterBlock - 1) / numThreadsPerClusterBlock;
#if BLOCK_SHARED_MEM_OPTIMIZATION
    const unsigned int clusterBlockSharedDataSize =
        numThreadsPerClusterBlock * sizeof(unsigned char) +
        numClusters * numCoords * sizeof(float);

    cudaDeviceProp deviceProp;
    int deviceNum;
    cudaGetDevice(&deviceNum);
    cudaGetDeviceProperties(&deviceProp, deviceNum);

    if (clusterBlockSharedDataSize > deviceProp.sharedMemPerBlock) {
        err("WARNING: Your CUDA hardware has insufficient block shared memory. "
            "You need to recompile with BLOCK_SHARED_MEM_OPTIMIZATION=0. "
            "See the README for details.\n");
    }
#else
    const unsigned int clusterBlockSharedDataSize =
        numThreadsPerClusterBlock * sizeof(unsigned char);
#endif

    const unsigned int numReductionThreads =
        nextPowerOfTwo(numClusterBlocks);
    const unsigned int reductionBlockSharedDataSize =
        numReductionThreads * sizeof(unsigned int);

    checkCuda(cudaMalloc(&deviceObjects, numObjs*numCoords*sizeof(float)));
    checkCuda(cudaMalloc(&deviceClusters, numClusters*numCoords*sizeof(float)));
    checkCuda(cudaMalloc(&deviceMembership, numObjs*sizeof(int)));
    checkCuda(cudaMalloc(&deviceIntermediates, numReductionThreads*sizeof(unsigned int)));

    checkCuda(cudaMemcpy(deviceObjects, dimObjects[0],
              numObjs*numCoords*sizeof(float), cudaMemcpyHostToDevice));
    checkCuda(cudaMemcpy(deviceMembership, membership,
              numObjs*sizeof(int), cudaMemcpyHostToDevice));

    do {
        checkCuda(cudaMemcpy(deviceClusters, dimClusters[0],
                  numClusters*numCoords*sizeof(float), cudaMemcpyHostToDevice));

        find_nearest_cluster
            <<< numClusterBlocks, numThreadsPerClusterBlock, clusterBlockSharedDataSize >>>
            (numCoords, numObjs, numClusters,
             deviceObjects, deviceClusters, deviceMembership, deviceIntermediates);

        cudaDeviceSynchronize(); checkLastCudaError();

        compute_delta <<< 1, numReductionThreads, reductionBlockSharedDataSize >>>
            (deviceIntermediates, numClusterBlocks, numReductionThreads);

        cudaDeviceSynchronize(); checkLastCudaError();

        int d;
        checkCuda(cudaMemcpy(&d, deviceIntermediates,
                  sizeof(int), cudaMemcpyDeviceToHost));
        delta = (float)d;

        checkCuda(cudaMemcpy(membership, deviceMembership,
                  numObjs*sizeof(int), cudaMemcpyDeviceToHost));

        for (i=0; i<numObjs; i++) {
            /* find the array index of nestest cluster center */
            index = membership[i];

            /* update new cluster centers : sum of objects located within */
            newClusterSize[index]++;
            for (j=0; j<numCoords; j++)
                newClusters[j][index] += objects[i][j];
        }

        //  TODO: Flip the nesting order
        //  TODO: Change layout of newClusters to [numClusters][numCoords]
        /* average the sum and replace old cluster centers with newClusters */
        for (i=0; i<numClusters; i++) {
            for (j=0; j<numCoords; j++) {
                if (newClusterSize[i] > 0)
                    dimClusters[j][i] = newClusters[j][i] / newClusterSize[i];
                newClusters[j][i] = 0.0;   /* set back to 0 */
            }
            newClusterSize[i] = 0;   /* set back to 0 */
        }

        delta /= numObjs;
    } while (delta > threshold && loop++ < 500);

    *loop_iterations = loop + 1;

    /* allocate a 2D space for returning variable clusters[] (coordinates
       of cluster centers) */
    malloc2D(clusters, numClusters, numCoords, float);
    for (i = 0; i < numClusters; i++) {
        for (j = 0; j < numCoords; j++) {
            clusters[i][j] = dimClusters[j][i];
        }
    }

    checkCuda(cudaFree(deviceObjects));
    checkCuda(cudaFree(deviceClusters));
    checkCuda(cudaFree(deviceMembership));
    checkCuda(cudaFree(deviceIntermediates));

    free(dimObjects[0]);
    free(dimObjects);
    free(dimClusters[0]);
    free(dimClusters);
    free(newClusters[0]);
    free(newClusters);
    free(newClusterSize);

    return clusters;
}






int main(){
    
int i,j;
int opt;
int    *pred;    /* [numObjs] */
float **objects;       /* [numObjs][numCoords] data objects */
float **clusters;      /* [numClusters][numCoords] cluster center */
float   threshold;
int     loop_iterations;



pred = (int*) malloc(numObjs * sizeof(int));
objects = (float**) malloc(numObjs * sizeof(float*));


FILE *fp;
fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128_label.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }

        for (int j = 0; j < numObjs; j++) {
            if (fscanf(fp, "%d", &label[j]) != 1) {
                printf("Failed to read matrix element\n");
                printf("%d",j);
                return 1;
            }
            if (j < numObjs - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
printf("标签文件读取结束读取结束\n");
fclose(fp);
short temp;
    fp = fopen("drive/MyDrive/cuda_kmeans_data/imag128.txt", "r");
    if (fp == NULL) {
        printf("Failed to open file\n");
        return 1;
    }
    for (int i = 0; i < numObjs; i++) {
        for (int j = 0; j < numCoords; j++) {
            if (fscanf(fp, "%hd", &temp) != 1) {
                
                printf("Failed to read matrix element\n");
                printf("%d,%d",i,j);
                return 1;
            }
            imgvec[i][j]=(float)(temp);
            if (j < numCoords - 1) {
                char c;
                if (fscanf(fp, "%c", &c) != 1 || c != ',') {
                    printf("Failed to read comma\n");
                    return 1;
                }
            }
        }
        if(i%1000==0) printf("已完成%d行读取\n",i);
    }
    printf("读取结束\n");
    fclose(fp);
  
    for(int i=0;i<numObjs;i++){
        objects[i]=(float*)&(imgvec[i]);
    }

}





/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(24): error: expected a ")"

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(30): warning #12-D: parsing restarts here after previous syntax error

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(36): error: identifier "objects" is undefined

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(36): error: identifier "objectId" is undefined

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(36): error: identifier "clusters" is undefined

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(36): error: identifier "clusterId" is undefined

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(45): error: expected a ")"

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(49): warning #12-D: parsing restarts here after previous syntax error

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34019f3060f9.cu(60): error: identifier "deviceClusters" is undefined

/tmp/tmpuoyegvyg/5f442c92-2e53-4da8-9484-34